Let's try tuning the prompt for `Qwen/Qwen2.5-0.5B-Instruct`.

In [1]:
from openai import OpenAI 

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="-"
)

In [40]:
client.models.list()

SyncPage[Model](data=[Model(id='Qwen/Qwen2.5-0.5B-Instruct', created=1756748005, object='model', owned_by='vllm', root='Qwen/Qwen2.5-0.5B-Instruct', parent=None, max_model_len=256, permission=[{'id': 'modelperm-36b7ea40bb12488fa10a333ec4fc9d22', 'object': 'model_permission', 'created': 1756748005, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}])], object='list')

In [41]:
def create_chat_completion(
    sys_prompt: str, 
    user_msg: str,
    model: str = 'Qwen/Qwen2.5-0.5B-Instruct',
    **kwargs 
):
    res = client.chat.completions.create(
        model=model,
        messages=[
            {
                'role': 'system',
                'content': sys_prompt
            },
            {
                'role': 'user',
                'content': user_msg
            }
        ],
        **kwargs
    )

    return res.choices[0].message.content

Load an exapmle dataset.

In [29]:
from mteb.tasks import NQ

nq = NQ() 
nq.load_data()

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4201/4201 [00:00<00:00, 23415.46 examples/s]


In [30]:
queries = list(nq.queries['test'].items())
corpus = list(nq.corpus['test'].items())
qrels = list(nq.relevant_docs['test'].items())

In [31]:
qrels[:5]

[('test0', {'doc0': 1, 'doc1': 1}),
 ('test1', {'doc6': 1}),
 ('test2', {'doc10': 1}),
 ('test3', {'doc17': 1, 'doc18': 1}),
 ('test4', {'doc42': 1})]

In [47]:
nq.queries['test']['test3']

'who is the leader of the ontario pc party'

In [48]:
nq.corpus['test']['doc17']

"Patrick Brown (politician) Patrick Walter Brown MPP (born May 26, 1978) is a Canadian politician who is the leader of the Progressive Conservative Party of Ontario and Ontario's Leader of the Official Opposition. Brown was a federal Conservative member of the House of Commons of Canada from 2006-15 representing the riding of Barrie."

In [49]:
sys_prompt = """
You are a helpful AI assistant. 
Given a document excerpt, generate a concise and highly relevant search question. 
Output only the question in lowercase, without quotes or ending punctuation.
""".strip()

In [50]:
create_chat_completion(
    sys_prompt,
    nq.corpus['test']['doc17'], 
    temperature=0
)

"What is Patrick Brown's political affiliation?"